In [3]:
import numpy as np
import sys
sys.path.append("../")
from algorithm.koopmanlib.dictionary import PsiNN

In [4]:
basis_function = PsiNN(layer_sizes=[100,100,100], n_psi_train=22)

In [5]:
from duffing_oscillator import duffing_oscillator
from scipy.integrate import solve_ivp

# Parameters
delta = 0.5
alpha = -1
beta = 1

# Sample N=1000 initial points, with uniform distribution [-2, 2].
N = 1000
np.random.seed(39)
sample_data_x0 = (np.random.rand(N, 2) * 4) - 2

tspan = [0, 2.5]
nstep = 11
data = np.zeros((N, 2, nstep))

t_eval = np.linspace(tspan[0], tspan[1], nstep)

for idx, x0 in enumerate(sample_data_x0):
    sol = solve_ivp(duffing_oscillator, tspan, x0, t_eval=t_eval, args=(delta, alpha, beta))
    data[idx, 0, :] = sol.y[0]
    data[idx, 1, :] = sol.y[1]

# Pairwise shifted data X and Y
X = data[:, :, :-1] # shape: (1000, 2, 10)
Y = data[:, :, 1:]

# Transpose and reshape the data.
X = np.transpose(X, (0, 2, 1)).reshape(N * (nstep-1), 2) # shape: (10000, 2)
Y = np.transpose(Y, (0, 2, 1)).reshape(N * (nstep-1), 2)

In [6]:
len_all = X.shape[0]
data_x_train = X[:int(0.7*len_all),:]
data_x_valid = X[int(0.7*len_all)+1:,:]

data_y_train = Y[:int(0.7*len_all),:]
data_y_valid = Y[int(0.7*len_all)+1:,:]

In [7]:
data_train = [data_x_train, data_y_train]
data_valid = [data_x_valid, data_y_valid]

In [6]:
# Using solver0
from algorithm.koopmanlib.solver_0 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=3000,
             batch_size=5000,
             lr=1e-4,
             log_interval=10,
             lr_decay_factor=.8)

Epoch 1/2
2/2 [==============================] - 1s 222ms/step - loss: 0.0029 - val_loss: 3.2400e-04
Epoch 2/2
2/2 [==============================] - 0s 40ms/step - loss: 0.0021 - val_loss: 0.0064
number of the outer loop: 0
Epoch 1/2
2/2 [==============================] - 0s 81ms/step - loss: 1.0212e-04 - val_loss: 1.5607e-04
Epoch 2/2
2/2 [==============================] - 0s 47ms/step - loss: 1.2967e-04 - val_loss: 1.2181e-04
number of the outer loop: 1
Epoch 1/2
2/2 [==============================] - 0s 71ms/step - loss: 9.6978e-05 - val_loss: 1.3346e-04
Epoch 2/2
2/2 [==============================] - 0s 39ms/step - loss: 1.1186e-04 - val_loss: 1.1559e-04
number of the outer loop: 2
Epoch 1/2
2/2 [==============================] - 0s 63ms/step - loss: 9.5070e-05 - val_loss: 1.2426e-04
Epoch 2/2
2/2 [==============================] - 0s 38ms/step - loss: 1.0207e-04 - val_loss: 1.1229e-04
number of the outer loop: 3
Epoch 1/2
2/2 [==============================] - 0s 63ms/step - los

In [7]:
# Results from solver0
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]
Psi_X, Psi_Y = solver.get_basis(data_train[0], data_train[1])

resDMD_DL_outputs0 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict,
    'Psi_X': Psi_X,
    'Psi_Y': Psi_Y
    }
}

np.save("solver_0_outputs.npy", resDMD_DL_outputs0, allow_pickle=True)

print(resDMD_DL_outputs0['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs0['resDMD_DL_outputs']['efuns'].shape)

[ 1.01149408e+00+0.00000000e+00j  9.99998604e-01+0.00000000e+00j
  8.32221769e-01+1.48168938e-01j  8.32221769e-01-1.48168938e-01j
  7.66872490e-01+4.11078123e-01j  7.66872490e-01-4.11078123e-01j
  6.31712487e-01-2.53131775e-01j  6.31712487e-01+2.53131775e-01j
  3.81106038e-01+4.87601599e-01j  3.81106038e-01-4.87601599e-01j
  1.89242576e-01+0.00000000e+00j  5.17772783e-02+0.00000000e+00j
  1.39184297e-02-6.06554255e-01j  1.39184297e-02+6.06554255e-01j
  4.78632417e-03+5.64509782e-02j  4.78632417e-03-5.64509782e-02j
  4.55324373e-03+1.75692995e-03j  4.55324373e-03-1.75692995e-03j
  1.66775325e-03+0.00000000e+00j  4.79961076e-04+1.13562734e-04j
  4.79961076e-04-1.13562734e-04j -5.41506440e-05+0.00000000e+00j
 -2.89915428e-04+0.00000000e+00j -5.48448282e-03+1.27921108e-02j
 -5.48448282e-03-1.27921108e-02j]
(10000, 25)


In [8]:
# Using solver1
from algorithm.koopmanlib.solver_1 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=3000,
             batch_size=5000,
             lr=1e-4,
             log_interval=10,
             lr_decay_factor=.8)

Epoch 1/2
2/2 [==============================] - 9s 1s/step - loss: 0.4358 - val_loss: 0.4470
Epoch 2/2
2/2 [==============================] - 0s 95ms/step - loss: 0.4259 - val_loss: 0.4141
number of the outer loop: 0
Epoch 1/2
2/2 [==============================] - 0s 135ms/step - loss: 0.4230 - val_loss: 0.4088
Epoch 2/2
2/2 [==============================] - 0s 106ms/step - loss: 0.4050 - val_loss: 0.4196
number of the outer loop: 1
Epoch 1/2
2/2 [==============================] - 0s 125ms/step - loss: 0.4068 - val_loss: 0.3905
Epoch 2/2
2/2 [==============================] - 0s 94ms/step - loss: 0.4153 - val_loss: 0.4256
number of the outer loop: 2
Epoch 1/2
2/2 [==============================] - 0s 121ms/step - loss: 0.3969 - val_loss: 0.4161
Epoch 2/2
2/2 [==============================] - 0s 97ms/step - loss: 0.3911 - val_loss: 0.4204
number of the outer loop: 3
Epoch 1/2
2/2 [==============================] - 0s 126ms/step - loss: 0.3857 - val_loss: 0.4162
Epoch 2/2
2/2 [======

In [9]:
# Results from solver1
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]
Psi_X, Psi_Y = solver.get_basis(data_train[0], data_train[1])

resDMD_DL_outputs1 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict,
    'Psi_X': Psi_X,
    'Psi_Y': Psi_Y
    }
}

np.save("solver_1_outputs.npy", resDMD_DL_outputs1, allow_pickle=True)

print(resDMD_DL_outputs1['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs1['resDMD_DL_outputs']['efuns'].shape)

[ 1.00215065+0.j          1.00000131+0.j          0.85031217-0.09992105j
  0.85031217+0.09992105j  0.83683044-0.28818534j  0.83683044+0.28818534j
  0.83284848+0.j          0.81635208-0.34956329j  0.81635208+0.34956329j
  0.76884865-0.22658829j  0.76884865+0.22658829j  0.62875644-0.4120418j
  0.62875644+0.4120418j   0.60895712+0.24268905j  0.60895712-0.24268905j
  0.44235142-0.47876883j  0.44235142+0.47876883j  0.34606756-0.27699206j
  0.34606756+0.27699206j  0.33865313-0.35014331j  0.33865313+0.35014331j
  0.16845987+0.j          0.09494151-0.60045198j  0.09494151+0.60045198j
 -0.00185898+0.j        ]
(10000, 25)


In [10]:
# Using solver2
from algorithm.koopmanlib.solver_2 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=3000,
             batch_size=5000,
             lr=1e-4,
             log_interval=10,
             lr_decay_factor=.8)

Epoch 1/2
2/2 [==============================] - 1s 220ms/step - loss: 349196.2143 - val_loss: 243467.4219
Epoch 2/2
2/2 [==============================] - 0s 63ms/step - loss: 315099.8973 - val_loss: 224549.7344
number of the outer loop: 0
Epoch 1/2
2/2 [==============================] - 0s 87ms/step - loss: 292006.8080 - val_loss: 208793.2656
Epoch 2/2
2/2 [==============================] - 0s 77ms/step - loss: 271298.1518 - val_loss: 195165.0156
number of the outer loop: 1
Epoch 1/2
2/2 [==============================] - 0s 75ms/step - loss: 255011.0491 - val_loss: 183018.8906
Epoch 2/2
2/2 [==============================] - 0s 53ms/step - loss: 238975.7054 - val_loss: 172251.7031
number of the outer loop: 2
Epoch 1/2
2/2 [==============================] - 0s 73ms/step - loss: 221345.1741 - val_loss: 168355.9062
Epoch 2/2
2/2 [==============================] - 0s 47ms/step - loss: 219903.0312 - val_loss: 163957.0469
number of the outer loop: 3
Epoch 1/2
2/2 [========================

In [11]:
# Results from solver2
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]
Psi_X, Psi_Y = solver.get_basis(data_train[0], data_train[1])

resDMD_DL_outputs2 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict,
    'Psi_X': Psi_X,
    'Psi_Y': Psi_Y
    }
}

np.save("solver_2_outputs.npy", resDMD_DL_outputs2, allow_pickle=True)

print(resDMD_DL_outputs2['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs2['resDMD_DL_outputs']['efuns'].shape)

[1.0000702 +0.j         0.90829184+0.14675882j 0.90829184-0.14675882j
 0.86377608+0.j         0.71937997+0.j         0.70471243+0.19391872j
 0.70471243-0.19391872j 0.70374664+0.32188612j 0.70374664-0.32188612j
 0.44340058-0.11319606j 0.44340058+0.11319606j 0.38201557+0.02059753j
 0.38201557-0.02059753j 0.37505679-0.49941996j 0.37505679+0.49941996j
 0.1910589 +0.2748614j  0.1910589 -0.2748614j  0.17282958-0.51262407j
 0.17282958+0.51262407j 0.10836425+0.02038279j 0.10836425-0.02038279j
 0.09004223+0.j         0.03139627+0.00943135j 0.03139627-0.00943135j
 0.01125819+0.j        ]
(10000, 25)


In [8]:
# Using solver3
from algorithm.koopmanlib.solver_3 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=50,
             batch_size=5000,
             lr=1e-4,
             log_interval=10,
             lr_decay_factor=.8)

Epoch 1/2
2/2 [==============================] - 4s 804ms/step - loss: 0.0011 - val_loss: 0.0014
Epoch 2/2
2/2 [==============================] - 0s 161ms/step - loss: 0.0019 - val_loss: 8.4881e-04
number of the outer loop: 0
Epoch 1/2
2/2 [==============================] - 0s 227ms/step - loss: 3.3023e-04 - val_loss: 0.0011
Epoch 2/2
2/2 [==============================] - 0s 104ms/step - loss: 9.0432e-04 - val_loss: 1.7530e-04
number of the outer loop: 1
Epoch 1/2
2/2 [==============================] - 0s 224ms/step - loss: 1.8603e-04 - val_loss: 8.6381e-04
Epoch 2/2
2/2 [==============================] - 0s 107ms/step - loss: 7.1646e-04 - val_loss: 9.4910e-05
number of the outer loop: 2
Epoch 1/2
2/2 [==============================] - 0s 214ms/step - loss: 1.2890e-04 - val_loss: 5.5107e-04
Epoch 2/2
2/2 [==============================] - 0s 106ms/step - loss: 4.7506e-04 - val_loss: 7.5482e-05
number of the outer loop: 3
Epoch 1/2
2/2 [==============================] - 0s 293ms/step -

In [9]:
# Results from solver3
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]
Psi_X, Psi_Y = solver.get_basis(data_train[0], data_train[1])

resDMD_DL_outputs3 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict,
    'Psi_X': Psi_X,
    'Psi_Y': Psi_Y
    }
}

np.save("solver_3_outputs.npy", resDMD_DL_outputs3, allow_pickle=True)

print(resDMD_DL_outputs3['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs3['resDMD_DL_outputs']['efuns'].shape)

[ 9.99999288e-01+0.00000000e+00j  9.80136178e-01+0.00000000e+00j
  8.73525063e-01+0.00000000e+00j  7.86124027e-01+2.19716474e-01j
  7.86124027e-01-2.19716474e-01j  5.84155758e-01+5.37582785e-01j
  5.84155758e-01-5.37582785e-01j  4.35520834e-01+0.00000000e+00j
  3.86733594e-01+0.00000000e+00j  2.67615707e-01-3.99852211e-01j
  2.67615707e-01+3.99852211e-01j  2.08520666e-01+0.00000000e+00j
  1.66095085e-02-6.92306590e-03j  1.66095085e-02+6.92306590e-03j
  6.57073332e-03+0.00000000e+00j  1.08307429e-03+0.00000000e+00j
  5.12233782e-04+1.29598687e-03j  5.12233782e-04-1.29598687e-03j
  9.22588499e-05+0.00000000e+00j  6.12218863e-05+3.30360472e-05j
  6.12218863e-05-3.30360472e-05j -4.26833143e-03+1.01047394e-03j
 -4.26833143e-03-1.01047394e-03j -4.45158255e-02-3.75089359e-01j
 -4.45158255e-02+3.75089359e-01j]
(10000, 25)


In [11]:
print(Psi_X.shape)

(7000, 25)


In [14]:
# Using solver4
from algorithm.koopmanlib.solver_4 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=3000,
             batch_size=5000,
             lr=1e-4,
             log_interval=10,
             lr_decay_factor=.8)

Epoch 1/2
2/2 [==============================] - 3s 550ms/step - loss: 0.2389 - val_loss: 0.1592
Epoch 2/2
2/2 [==============================] - 0s 77ms/step - loss: 0.1859 - val_loss: 0.1291
number of the outer loop: 0
Epoch 1/2
2/2 [==============================] - 0s 95ms/step - loss: 0.1670 - val_loss: 0.1083
Epoch 2/2
2/2 [==============================] - 0s 69ms/step - loss: 0.1481 - val_loss: 0.0948
number of the outer loop: 1
Epoch 1/2
2/2 [==============================] - 0s 92ms/step - loss: 0.1334 - val_loss: 0.0845
Epoch 2/2
2/2 [==============================] - 0s 70ms/step - loss: 0.1350 - val_loss: 0.0868
number of the outer loop: 2
Epoch 1/2
2/2 [==============================] - 0s 93ms/step - loss: 0.1166 - val_loss: 0.0858
Epoch 2/2
2/2 [==============================] - 0s 68ms/step - loss: 0.1091 - val_loss: 0.0805
number of the outer loop: 3
Epoch 1/2
2/2 [==============================] - 0s 87ms/step - loss: 0.1071 - val_loss: 0.0746
Epoch 2/2
2/2 [========

In [15]:
# Results from solver4
evalues = solver.eigenvalues.T
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode().T
N_dict = np.shape(evalues)[0]
Psi_X, Psi_Y = solver.get_basis(data_train[0], data_train[1])

resDMD_DL_outputs4 = {
    'resDMD_DL_outputs':{
    'efuns': efuns,
    'evalues': evalues,
    'kpm_modes': kpm_modes,
    'N_dict':N_dict,
    'Psi_X': Psi_X,
    'Psi_Y': Psi_Y
    }
}

np.save("solver_4_outputs.npy", resDMD_DL_outputs4, allow_pickle=True)

print(resDMD_DL_outputs4['resDMD_DL_outputs']['evalues'])
print(resDMD_DL_outputs4['resDMD_DL_outputs']['efuns'].shape)

[1.00856603+0.j         1.0000824 +0.j         0.84485195+0.15954606j
 0.84485195-0.15954606j 0.80849348+0.j         0.77099853+0.36686792j
 0.77099853-0.36686792j 0.76560429+0.43102743j 0.76560429-0.43102743j
 0.61415773-0.20841092j 0.61415773+0.20841092j 0.41833158+0.49979625j
 0.41833158-0.49979625j 0.06415889-0.66583368j 0.06415889+0.66583368j
 0.03437756+0.j         0.02241227+0.01193446j 0.02241227-0.01193446j
 0.01862054+0.01517557j 0.01862054-0.01517557j 0.01630185+0.j
 0.00718216-0.004282j   0.00718216+0.004282j   0.0024481 +0.02438504j
 0.0024481 -0.02438504j]
(10000, 25)
